In [1]:
import pandas as pd
import numpy as np
import warnings
import zipfile
import os

In [2]:
warnings.filterwarnings("ignore")
data_path = 'C:/Users/forqu/OneDrive/Documentos/IronHack/Github/Projetos/2022-Better-Working-World-Data-Challenge/notebooks/'
output_path = 'C:/Users/forqu/OneDrive/Documentos/IronHack/Github/Projetos/2022-Better-Working-World-Data-Challenge/notebooks/dataset./output/'

In [3]:
if not os.path.exists(data_path+'training_data/'):
    os.mkdir(data_path+'/training_data/')
    with zipfile.ZipFile(data_path+'GBIF_training_data.zip', 'r') as zip_ref:
        zip_ref.extractall(data_path+'/training_data/')

In [81]:
data = pd.read_csv(data_path+'training_data/occurrence.txt', sep='\t', parse_dates=['eventDate'])
data.to_csv('frogs.csv', index=False)

In [ ]:
columns = [
    'gbifID','basisOfRecord','eventDate','country','continent','stateProvince',
    'decimalLatitude','decimalLongitude','species', 'coordinateUncertaintyInMeters'
]

country_names = {
    'AU':'Australia', 'CR':'Costa Rica', 'ZA':'South Africa','MX':'Mexico','HN':'Honduras',
    'MZ':'Mozambique','BW':'Botswana','MW':'Malawi','CO':'Colombia','PA':'Panama','NI':'Nicaragua',
    'BZ':'Belize','ZW':'Zimbabwe','SZ':'Eswatini','ZM':'Zambia','GT':'Guatemala','LS':'Lesotho',
    'SV':'El Salvador', 'AO':'Angola', np.nan:'unknown or invalid'
}

continent_names = {
    'AU':'Australia', 'CR':'Central America', 'ZA':'Africa','MX':'Central America','HN':'Central America',
    'MZ':'Africa','BW':'Africa','MW':'Africa','CO':'Central America','PA':'Central America',
    'NI':'Central America','BZ':'Central America','ZW':'Africa','SZ':'Africa','ZM':'Africa',
    'GT':'Central America','LS':'Africa','SV':'Central America','AO':'Africa', np.nan:'unknown or invalid' 
}

In [55]:
data = (
    pd.read_csv(data_path+'training_data/occurrence.txt', sep='\t', parse_dates=['eventDate']
               )
    .assign(
        country =  lambda x: x.countryCode.map(country_names),
        continent =  lambda x: x.countryCode.map(continent_names),
        species = lambda x: x.species.str.title(),
        stateProvince = lambda x: x.stateProvince.str.title()
    )    
    # Rename ranoidea australis to cyclorana australis
    .assign(species = lambda x: np.where(x.species == 'Ranoidea Australis', 'Cyclorana Australis', x.species))
    [columns]
)

In [56]:
np.sum(data.isna())/len(data)*100

gbifID                           0.000000
basisOfRecord                    0.000000
eventDate                        4.522914
country                          0.000000
continent                        0.000000
stateProvince                    0.357086
decimalLatitude                  0.000000
decimalLongitude                 0.000000
species                          0.000000
coordinateUncertaintyInMeters    8.986485
dtype: float64

In [57]:
data.head(10)

,gbifID,basisOfRecord,eventDate,country,continent,stateProvince,decimalLatitude,decimalLongitude,species,coordinateUncertaintyInMeters
0,2574007008,HUMAN_OBSERVATION,2020-01-23 01:38:00,Australia,Australia,New South Wales,-32.719457,152.159267,Litoria Fallax,NaN
1,3457021422,HUMAN_OBSERVATION,2010-03-14 22:23:00,Costa Rica,Central America,Puntarenas,8.496999,-83.318613,Agalychnis Callidryas,12618.0
2,1571195309,HUMAN_OBSERVATION,2014-04-11 19:51:57,Costa Rica,Central America,Heredia,10.450801,-84.068659,Agalychnis Callidryas,NaN
3,2265778539,HUMAN_OBSERVATION,2018-07-21 21:24:21,Costa Rica,Central America,Puntarenas,9.756705,-84.613363,Agalychnis Callidryas,NaN
4,3407962690,HUMAN_OBSERVATION,2021-10-29 13:57:00,Australia,Australia,Queensland,-26.714302,152.815096,Litoria Fallax,977.0
5,2251945867,HUMAN_OBSERVATION,2018-05-27 20:31:39,Panama,Central America,Bocas Del Toro,9.517162,-82.580823,Agalychnis Callidryas,1395.0
6,1836625681,HUMAN_OBSERVATION,2018-02-25 06:17:00,Mozambique,Africa,Maputo,-25.077627,32.065052,Chiromantis Xerampelina,50.0
7,1944370584,HUMAN_OBSERVATION,2018-04-18 19:05:00,Australia,Australia,New South Wales,-33.693144,151.320884,Litoria Fallax,28930.0
8,1500319380,HUMAN_OBSERVATION,2009-07-28 17:16:00,Australia,Australia,Queensland,-27.888019,153.309342,Litoria Fallax,20674.0
9,1913177945,HUMAN_OBSERVATION,2016-01-28 11:55:00,Costa Rica,Central America,Limón,10.549700,-83.506400,Agalychnis Callidryas,NaN


In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193791 entries, 0 to 193790
Data columns (total 10 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   gbifID                         193791 non-null  int64         
 1   basisOfRecord                  193791 non-null  object        
 2   eventDate                      185026 non-null  datetime64[ns]
 3   country                        193791 non-null  object        
 4   continent                      193791 non-null  object        
 5   stateProvince                  193099 non-null  object        
 6   decimalLatitude                193791 non-null  float64       
 7   decimalLongitude               193791 non-null  float64       
 8   species                        193791 non-null  object        
 9   coordinateUncertaintyInMeters  176376 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 

In [83]:
data['coordinateUncertaintyInMeters'].quantile(q=[0.0,0.001,0.005,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.15,0.20])() 

count    1.763760e+05
mean     1.198690e+03
std      1.088191e+04
min      1.000000e-02
25%      9.000000e+00
50%      6.500000e+01
75%      2.000000e+02
max      1.601006e+06
Name: coordinateUncertaintyInMeters, dtype: float64

In [77]:
data[data['stateProvince'].isna()].groupby('country').count()

,gbifID,basisOfRecord,eventDate,continent,stateProvince,decimalLatitude,decimalLongitude,species,coordinateUncertaintyInMeters
country,,,,,,,,,
Australia,25,25,21,25,0,25,25,25,9
Botswana,14,14,12,14,0,14,14,14,1
Colombia,1,1,1,1,0,1,1,1,1
Costa Rica,289,289,269,289,0,289,289,289,264
Eswatini,50,50,50,50,0,50,50,50,1
Guatemala,2,2,2,2,0,2,2,2,0
Honduras,3,3,2,3,0,3,3,3,1
Lesotho,9,9,9,9,0,9,9,9,0
Malawi,3,3,1,3,0,3,3,3,0


In [79]:
data[data['country'] == 'unknown or invalid'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 85347 to 193583
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   gbifID                         103 non-null    int64         
 1   basisOfRecord                  103 non-null    object        
 2   eventDate                      81 non-null     datetime64[ns]
 3   country                        103 non-null    object        
 4   continent                      103 non-null    object        
 5   stateProvince                  80 non-null     object        
 6   decimalLatitude                103 non-null    float64       
 7   decimalLongitude               103 non-null    float64       
 8   species                        103 non-null    object        
 9   coordinateUncertaintyInMeters  21 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 8.9+ KB


In [59]:
data['species'].unique()

array(['Litoria Fallax', 'Agalychnis Callidryas',
       'Chiromantis Xerampelina', 'Dendrobates Auratus',
       'Crinia Signifera', 'Xenopus Laevis', 'Cyclorana Australis',
       'Austrochaperina Pluvialis', 'Crinia Glauerti'], dtype=object)

In [60]:
data[data['species']=='Litoria Fallax'].groupby(['species','country','stateProvince']).min()

gbifID  \
species        country   stateProvince                              
Litoria Fallax Australia Australian Capital Territory  2992302425   
                         New South Wales                223781775   
                         Northern Territory            1632954406   
                         Queensland                     223781770   
                         Victoria                      1272468087   

                                                           basisOfRecord  \
species        country   stateProvince                                     
Litoria Fallax Australia Australian Capital Territory  HUMAN_OBSERVATION   
                         New South Wales               HUMAN_OBSERVATION   
                         Northern Territory            HUMAN_OBSERVATION   
                         Queensland                    HUMAN_OBSERVATION   
                         Victoria                      HUMAN_OBSERVATION   

                                                                eventDate  \
species        country   stateProvince                                      
Litoria Fallax Australia Australian Capital Territory 2018-01-08 00:00:00   
                         New South Wales              1900-01-01 00:00:00   
                         Northern Territory           2010-02-19 10:00:00   
                         Queensland                   1770-05-23 00:00:00   
                         Victoria                     1999-10-06 00:00:00   

                                                       continent  \
species        country   stateProvince                             
Litoria Fallax Australia Australian Capital Territory  Australia   
                         New South Wales               Australia   
                         Northern Territory            Australia   
                         Queensland                    Australia   
                         Victoria                      Australia   

                                                       decimalLatitude  \
species        country   stateProvince                                   
Litoria Fallax Australia Australian Capital Territory       -35.311085   
                         New South Wales                    -37.201324   
                         Northern Territory                 -25.700000   
                         Queensland                         -29.123968   
                         Victoria                           -38.159100   

                                                       decimalLongitude  \
species        country   stateProvince                                    
Litoria Fallax Australia Australian Capital Territory        149.118015   
                         New South Wales                     145.550187   
                         Northern Territory                  134.100000   
                         Queensland                          144.295278   
                         Victoria                            143.784035   

                                                       coordinateUncertaintyInMeters  
species        country   stateProvince                                                
Litoria Fallax Australia Australian Capital Territory                            4.0  
                         New South Wales                                         1.0  
                         Northern Territory                                      NaN  
                         Queensland                                              1.0  
                         Victoria                                                2.0

In [61]:
data['country'].unique()

array(['Australia', 'Costa Rica', 'Panama', 'Mozambique', 'South Africa',
       'Botswana', 'Malawi', 'Colombia', 'Zimbabwe', 'Nicaragua',
       'Honduras', 'Belize', 'Mexico', 'Zambia', 'Guatemala', 'Eswatini',
       'unknown or invalid', 'Lesotho', 'Angola'], dtype=object)

In [62]:
data['decimalLatitude'].describe()

count    193791.000000
mean        -32.581960
std           7.801278
min         -43.766700
25%         -36.000000
50%         -33.835933
75%         -32.289586
max          21.349444
Name: decimalLatitude, dtype: float64

In [63]:
data['decimalLongitude'].describe()

count    193791.000000
mean        140.655139
std          35.361322
min         -99.221738
25%         144.589529
50%         149.974250
75%         151.194731
max         153.636442
Name: decimalLongitude, dtype: float64

In [64]:
data[['country','decimalLatitude','decimalLongitude']].groupby('country').min()

,decimalLatitude,decimalLongitude
country,,
Angola,-17.573000,23.189000
Australia,-43.766700,114.916670
Belize,17.150000,-88.733700
Botswana,-25.202000,21.969309
Colombia,6.296312,-77.563477
Costa Rica,8.367381,-85.480000
Eswatini,-27.250000,30.966670
Guatemala,13.965149,-90.866739
Honduras,13.880000,-88.930000


In [65]:
data[['country','decimalLatitude','decimalLongitude']].groupby('country').max()

,decimalLatitude,decimalLongitude
country,,
Angola,-17.535000,23.227000
Australia,-11.120000,153.636442
Belize,18.066670,-88.550000
Botswana,-17.794175,28.871913
Colombia,8.657574,-77.064222
Costa Rica,10.991875,-82.564591
Eswatini,-25.816670,32.016670
Guatemala,17.237000,-88.613861
Honduras,15.920000,-83.557800


In [66]:
data[data['eventDate'].isna()]

,gbifID,basisOfRecord,eventDate,country,continent,stateProvince,decimalLatitude,decimalLongitude,species,coordinateUncertaintyInMeters
57,1100522507,PRESERVED_SPECIMEN,NaT,Australia,Australia,New South Wales,-34.600000,150.533000,Crinia Signifera,10000.0
65,1100495416,PRESERVED_SPECIMEN,NaT,Australia,Australia,New South Wales,-30.733000,152.983000,Crinia Signifera,10000.0
72,1100497076,PRESERVED_SPECIMEN,NaT,Australia,Australia,New South Wales,-37.450000,149.966000,Crinia Signifera,10000.0
83,1100413609,PRESERVED_SPECIMEN,NaT,Australia,Australia,New South Wales,-37.016000,149.383000,Crinia Signifera,10000.0
103,1100494282,PRESERVED_SPECIMEN,NaT,Australia,Australia,New South Wales,-36.450000,148.266000,Crinia Signifera,10000.0
...,...,...,...,...,...,...,...,...,...,...
193773,3025667235,PRESERVED_SPECIMEN,NaT,South Africa,Africa,NaN,-25.000000,30.000000,Xenopus Laevis,1000.0
193774,3025667260,PRESERVED_SPECIMEN,NaT,Australia,Australia,NaN,-20.000000,145.000000,Cyclorana Australis,1000.0
193782,1322420222,PRESERVED_SPECIMEN,NaT,Costa Rica,Central America,NaN,10.134466,-83.507158,Dendrobates Auratus,22845.0
193783,1317939743,PRESERVED_SPECIMEN,NaT,Australia,Australia,New South Wales,-33.476700,150.467000,Crinia Signifera,5914.0


In [67]:
data['eventDate'].dt.year.describe()

count    185026.000000
mean       2009.650790
std          15.967347
min        1754.000000
25%        2004.000000
50%        2018.000000
75%        2019.000000
max        2022.000000
Name: eventDate, dtype: float64

In [68]:
data[data['eventDate'].dt.year >= 1990].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168148 entries, 0 to 193790
Data columns (total 10 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   gbifID                         168148 non-null  int64         
 1   basisOfRecord                  168148 non-null  object        
 2   eventDate                      168148 non-null  datetime64[ns]
 3   country                        168148 non-null  object        
 4   continent                      168148 non-null  object        
 5   stateProvince                  167717 non-null  object        
 6   decimalLatitude                168148 non-null  float64       
 7   decimalLongitude               168148 non-null  float64       
 8   species                        168148 non-null  object        
 9   coordinateUncertaintyInMeters  159537 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 

In [69]:
data[(data['eventDate'].dt.year < 1990)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16878 entries, 54 to 193788
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   gbifID                         16878 non-null  int64         
 1   basisOfRecord                  16878 non-null  object        
 2   eventDate                      16878 non-null  datetime64[ns]
 3   country                        16878 non-null  object        
 4   continent                      16878 non-null  object        
 5   stateProvince                  16754 non-null  object        
 6   decimalLatitude                16878 non-null  float64       
 7   decimalLongitude               16878 non-null  float64       
 8   species                        16878 non-null  object        
 9   coordinateUncertaintyInMeters  10168 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 1.4+ MB


In [70]:
data[(data['eventDate'].dt.year < 1990) | (data['eventDate'].isna())].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25643 entries, 54 to 193788
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   gbifID                         25643 non-null  int64         
 1   basisOfRecord                  25643 non-null  object        
 2   eventDate                      16878 non-null  datetime64[ns]
 3   country                        25643 non-null  object        
 4   continent                      25643 non-null  object        
 5   stateProvince                  25382 non-null  object        
 6   decimalLatitude                25643 non-null  float64       
 7   decimalLongitude               25643 non-null  float64       
 8   species                        25643 non-null  object        
 9   coordinateUncertaintyInMeters  16839 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 2.2+ MB


In [71]:
data[~((data['eventDate'].dt.year < 1990) | (data['eventDate'].isna()))].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168148 entries, 0 to 193790
Data columns (total 10 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   gbifID                         168148 non-null  int64         
 1   basisOfRecord                  168148 non-null  object        
 2   eventDate                      168148 non-null  datetime64[ns]
 3   country                        168148 non-null  object        
 4   continent                      168148 non-null  object        
 5   stateProvince                  167717 non-null  object        
 6   decimalLatitude                168148 non-null  float64       
 7   decimalLongitude               168148 non-null  float64       
 8   species                        168148 non-null  object        
 9   coordinateUncertaintyInMeters  159537 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 

In [72]:
pd.to_datetime(data['eventDate']).describe(datetime_is_numeric=True)

count                           185026
mean     2010-03-29 16:58:06.091036160
min                1754-03-01 00:00:00
25%                2004-09-12 00:00:00
50%                2018-06-25 00:00:00
75%                2019-11-17 00:00:00
max                2022-02-10 14:32:00
Name: eventDate, dtype: object

In [73]:
pd.to_datetime(data['eventDate']).quantile(q=[0.0,0.001,0.005,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.15,0.20])

0.000   1754-03-01
0.001   1900-01-01
0.005   1948-02-14
0.010   1960-09-15
0.020   1963-08-23
0.030   1968-05-12
0.040   1972-06-18
0.050   1976-06-14
0.060   1977-03-15
0.070   1979-11-08
0.080   1985-09-15
0.090   1989-09-14
0.100   1991-11-01
0.150   1997-09-17
0.200   2001-03-05
Name: eventDate, dtype: datetime64[ns]

In [74]:
data['eventDate'].describe()

count                  185026
unique                  18147
top       2010-12-17 00:00:00
freq                     1712
first     1754-03-01 00:00:00
last      2022-02-10 14:32:00
Name: eventDate, dtype: object